<a href="https://colab.research.google.com/github/mehdimerbah/COVID19_fake_news_detection/blob/main/models/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library and Data Import

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense

In [3]:
dataset = pd.read_csv('https://raw.githubusercontent.com/mehdimerbah/COVID19_fake_news_detection/main/preprocessing/whole_processed_data.csv')

In [4]:
oh_representation = [one_hot(terms, 5000) for terms in dataset['tweet']]

In [7]:
sentence_len = 400

embedded_data = pad_sequences(oh_representation, padding='pre', maxlen= sentence_len)

In [8]:
embedded_data[:5]

array([[   0,    0,    0, ..., 4424, 2850, 4175],
       [   0,    0,    0, ..., 2683, 1996, 4697],
       [   0,    0,    0, ..., 3009, 3460, 3711],
       [   0,    0,    0, ..., 1152, 2158, 2635],
       [   0,    0,    0, ..., 3972, 1490, 4541]], dtype=int32)

In [37]:
embedded_DF = pd.DataFrame(embedded_data)
embedded_DF['label'] = dataset['label']
embedded_DF['label'] = embedded_DF.label.eq('real').mul(1)
embedded_DF.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,label
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2587,648,4712,4268,4697,3023,3294,4801,3994,2201,4111,3564,707,4801,4075,2460,4207,4424,2850,4175,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4192,2683,1944,4697,3564,2597,2538,2255,1726,4192,2683,1996,4697,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,657,3332,1352,3648,1575,3676,751,1963,2466,3009,3460,3711,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3621,4703,2266,1490,449,2480,291,882,4064,2235,2127,2437,4742,1849,1152,2158,2635,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4338,4192,858,1546,1209,3994,1924,2132,4235,2542,69,4175,3845,1667,4192,3186,4235,2542,69,658,4458,1063,3818,4362,328,4511,1982,3972,1490,4541,1


In [38]:
training_set = embedded_DF.iloc[:6420]
validation_set = embedded_DF.iloc[6420:8560]
testing_set = embedded_DF.iloc[8560:]

In [39]:
validation_set.tail()
#testing_set.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,label
8555,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,4890,2205,3611,3751,2132,3745,3628,2878,2266,3945,4235,2057,4775,2683,4249,2132,4235,3129,2683,4108,2132,4235,2542,2057,2488,882,0
8556,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4644,4691,3516,4876,4316,4265,4265,2167,4677,4218,4265,955,3947,4937,4367,4876,1800,1
8557,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,551,598,3379,1405,1206,1890,983,1337,1572,652,2850,3379,1405,1206,1890,0
8558,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1209,4902,2266,3945,1255,3076,2850,1594,0
8559,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,469,1358,2132,3460,4235,1615,4852,117,2469,4071,2937,3160,1261,684,2132,4235,2683,2253,2082,2703,3824,2460,1889,2948,3139,1


In [40]:
# initializing model
model = Sequential()
# adding embedding layer
model.add(Embedding(5000, 100, input_length=sentence_len))
model.add(LSTM(150))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 400, 100)          500000    
                                                                 
 lstm_1 (LSTM)               (None, 150)               150600    
                                                                 
 dense_1 (Dense)             (None, 1)                 151       
                                                                 
Total params: 650,751
Trainable params: 650,751
Non-trainable params: 0
_________________________________________________________________


In [43]:
#training_set.iloc[:,:400]
training_set.iloc[:,400]

0       1
1       1
2       0
3       1
4       1
       ..
6415    0
6416    0
6417    0
6418    0
6419    1
Name: label, Length: 6420, dtype: int64

In [44]:
model_history = model.fit(training_set.iloc[:,:400], training_set.iloc[:,400], validation_data=(validation_set.iloc[:,:400], validation_set.iloc[:,400] ), epochs=10, batch_size=64)

Epoch 1/10
101/101 [==============================] - 111s 1s/step - loss: 0.3815 - accuracy: 0.8282 - val_loss: 0.2182 - val_accuracy: 0.9103
Epoch 2/10
101/101 [==============================] - 108s 1s/step - loss: 0.1472 - accuracy: 0.9456 - val_loss: 0.2416 - val_accuracy: 0.9000
Epoch 3/10
101/101 [==============================] - 108s 1s/step - loss: 0.1196 - accuracy: 0.9687 - val_loss: 0.2332 - val_accuracy: 0.9145
Epoch 4/10
101/101 [==============================] - 109s 1s/step - loss: 0.0455 - accuracy: 0.9855 - val_loss: 0.3007 - val_accuracy: 0.9121
Epoch 5/10
101/101 [==============================] - 109s 1s/step - loss: 0.0326 - accuracy: 0.9905 - val_loss: 0.3161 - val_accuracy: 0.9140
Epoch 6/10
101/101 [==============================] - 109s 1s/step - loss: 0.0148 - accuracy: 0.9967 - val_loss: 0.3914 - val_accuracy: 0.9042
Epoch 7/10
101/101 [==============================] - 110s 1s/step - loss: 0.0142 - accuracy: 0.9963 - val_loss: 0.4209 - val_accuracy: 0.8930

In [46]:
predictions = model.predict(validation_set.iloc[:,:400])

In [50]:
predictions

array([[1.9494952e-05],
       [9.9897122e-01],
       [3.5098195e-04],
       ...,
       [1.9328594e-03],
       [1.0089904e-02],
       [9.9997818e-01]], dtype=float32)

In [53]:
predictions = (model.predict(validation_set.iloc[:,:400]) > 0.5).astype("int32")

In [54]:
predictions

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [1]], dtype=int32)

In [56]:
round(accuracy_score(validation_set['label'], predictions), 5)

0.89907